In [20]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler


In [11]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [12]:
train['HF1'] = train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Fire_Points']
train['HF2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Fire_Points'])
train['HR1'] = abs(train['Horizontal_Distance_To_Hydrology']+train['Horizontal_Distance_To_Roadways'])
train['HR2'] = abs(train['Horizontal_Distance_To_Hydrology']-train['Horizontal_Distance_To_Roadways'])
train['FR1'] = abs(train['Horizontal_Distance_To_Fire_Points']+train['Horizontal_Distance_To_Roadways'])
train['FR2'] = abs(train['Horizontal_Distance_To_Fire_Points']-train['Horizontal_Distance_To_Roadways'])
train['ele_vert'] = train.Elevation-train.Vertical_Distance_To_Hydrology

train['slope_hyd'] = (train['Horizontal_Distance_To_Hydrology']**2+train['Vertical_Distance_To_Hydrology']**2)**0.5
train.slope_hyd=train.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
train['Mean_Amenities']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology + train.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
train['Mean_Fire_Hyd']=(train.Horizontal_Distance_To_Fire_Points + train.Horizontal_Distance_To_Hydrology) / 2

Feature Creation for test

In [13]:
test['HF1'] = test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Fire_Points']
test['HF2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Fire_Points'])
test['HR1'] = abs(test['Horizontal_Distance_To_Hydrology']+test['Horizontal_Distance_To_Roadways'])
test['HR2'] = abs(test['Horizontal_Distance_To_Hydrology']-test['Horizontal_Distance_To_Roadways'])
test['FR1'] = abs(test['Horizontal_Distance_To_Fire_Points']+test['Horizontal_Distance_To_Roadways'])
test['FR2'] = abs(test['Horizontal_Distance_To_Fire_Points']-test['Horizontal_Distance_To_Roadways'])
test['ele_vert'] = test.Elevation-test.Vertical_Distance_To_Hydrology

test['slope_hyd'] = (test['Horizontal_Distance_To_Hydrology']**2+test['Vertical_Distance_To_Hydrology']**2)**0.5
test.slope_hyd=test.slope_hyd.map(lambda x: 0 if np.isinf(x) else x) # remove infinite value if any

#Mean distance to Amenities 
test['Mean_Amenities']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology + test.Horizontal_Distance_To_Roadways) / 3 
#Mean Distance to Fire and Water 
test['Mean_Fire_Hyd']=(test.Horizontal_Distance_To_Fire_Points + test.Horizontal_Distance_To_Hydrology) / 2

In [14]:
X_train = train.drop(['Id','Cover_Type'],1)
Y_train = train['Cover_Type']
X_test = test.drop(['Id'],1)

In [22]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [23]:
X_train_tf.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,HF1,HF2,HR1,HR2,FR1,FR2,ele_vert,slope_hyd,Mean_Amenities,Mean_Fire_Hyd
0,-0.367095,-0.959980,-1.597132,0.146639,-0.834074,-0.908681,0.271454,0.571653,0.281259,4.334805,...,4.165194,4.390517,-0.848236,-0.959357,1.702247,5.505993,-0.246574,0.102333,1.673476,4.165194
1,-0.381461,-0.914559,-1.715424,-0.072337,-0.932054,-0.999246,0.238732,0.703225,0.346627,4.285710,...,4.078395,4.383090,-0.968254,-1.016605,1.619137,5.580595,-0.246574,-0.110745,1.571055,4.078395
2,0.130912,-0.160577,-0.887379,0.194243,0.227369,1.106379,0.696843,0.834797,-0.002005,4.191156,...,4.036731,4.234547,1.089401,1.098462,2.902092,2.309390,0.098281,0.184798,2.847589,4.036731
3,0.085421,-0.015231,0.177250,0.070474,1.092853,1.038455,0.827731,0.834797,-0.285268,4.272981,...,4.092283,4.342240,1.005533,1.048950,2.902092,2.512851,-0.075352,0.154476,2.835485,4.092283
4,-0.369489,-1.014485,-1.715424,-0.353198,-0.850404,-0.998491,0.238732,0.659368,0.324838,4.237524,...,3.981180,4.388660,-1.010188,-0.970188,1.594299,5.519557,-0.246574,-0.384926,1.519379,3.981180


### Logistic regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,f1_score


In [5]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [ ]:
lr = LogisticRegression(solver='liblinear',max_iter=1000,penalty='l2',random_state=100)
lr.fit(X_train_tf,Y_train)
Y_train_pred = lr.predict(X_train_tf)

In [ ]:
lr.fit(X_train,Y_train)
Y_train_pred = lr.predict(X_train)
print(np.unique(Y_train_pred))
Y_test_pred = lr.predict(X_test)
print(np.unique(Y_test_pred))


In [ ]:
print(accuracy_score(Y_train,Y_train_pred))
Y_test_pred = lr.predict(X_test_tf)

In [ ]:
print(np.unique(Y_test_pred))
print(np.unique(Y_train_pred))
print(len(Y_test_pred))

In [ ]:
sample_submission['Cover_Type'] = Y_test_pred

In [ ]:
sample_submission.to_csv('submission_lr.csv',index=False)

### Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif


In [ ]:
feature_selector = SelectKBest(f_classif, k=50)
X_kbest = pd.DataFrame(feature_selector.fit_transform(X_train_tf, Y_train))

In [ ]:
lr.fit(X_kbest,Y_train)
Y_train_pred = lr.predict(X_kbest)
print(accuracy_score(Y_train,Y_train_pred))

In [ ]:
import sklearn.ensemble

### Decision Tree classifier

In [6]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train_tf,Y_train)
Y_train_pred_dt = dtc.predict(X_train_tf)

In [7]:
Y_train_pred_dt

array([5, 5, 2, ..., 3, 3, 3])

### Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [7]:
results_rf = pd.DataFrame(data=None,columns=['n_estimator','criterion','max_depth',
                                            'accuracy_min','accuracy_max','accuracy_mean'])

In [31]:
N_estimators = [20,40,60,80]
Criterion = ['gini','entropy']
Max_depth = [20,30,40,50,60,70]
i=0
for n_estimator in N_estimators:
    for criterion in Criterion:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'criterion'] = criterion
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion=criterion,
                                                      max_depth=max_depth,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1

In [35]:
RandomForestClassifier?

In [1]:
results_rf

NameError: name 'results_rf' is not defined

In [32]:
results_rf.sort_values('accuracy_mean')

,n_estimator,criterion,max_depth,accuracy_min,accuracy_max,accuracy_mean
6,20,entropy,20,0.715278,0.830026,0.761971
0,20,gini,20,0.724868,0.832011,0.762765
8,20,entropy,40,0.727513,0.830357,0.766204
10,20,entropy,60,0.727513,0.830357,0.766402
9,20,entropy,50,0.727513,0.830357,0.766402
11,20,entropy,70,0.727513,0.830357,0.766402
7,20,entropy,30,0.733796,0.833003,0.769511
12,40,gini,20,0.731812,0.835648,0.770304
4,20,gini,60,0.724868,0.842262,0.771429
3,20,gini,50,0.724868,0.842262,0.771429


In [39]:
results_rf=pd.DataFrame(None,columns=['n_estimator','max_features','max_depth',
                                            'accuracy_min','accuracy_max','accuracy_mean'])

N_estimators = [80,100,120]
Max_depth = [50,60,70,80]
Max_features = ["auto",'log2',None,20]
i=0
for n_estimator in N_estimators:
    for max_features in Max_features:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'max_features'] = max_features
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion='entropy',
                                                      max_depth=max_depth,
                                                      max_features=max_features,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1

In [40]:
results_rf.sort_values('accuracy_mean')

,n_estimator,max_features,max_depth,accuracy_min,accuracy_max,accuracy_mean
4,80,log2,50,0.734458,0.839286,0.773347
5,80,log2,60,0.734458,0.839286,0.773347
6,80,log2,70,0.734458,0.839286,0.773347
7,80,log2,80,0.734458,0.839286,0.773347
23,100,log2,80,0.734458,0.837632,0.773677
22,100,log2,70,0.734458,0.837632,0.773677
21,100,log2,60,0.734458,0.837632,0.773677
20,100,log2,50,0.734458,0.837632,0.773677
39,120,log2,80,0.732143,0.837963,0.775529
38,120,log2,70,0.732143,0.837963,0.775529


Best hyperparameters

max_depth = 50;
criterion = entropy;
n_estimators = 120;
max_feature = 20

In [24]:
# parameter that give best score so far on test - 0.77105
# n_estimators=150,max_depth=50,max_features=20,criterion='entropy',max_features=20
r1 = cross_val_score(RandomForestClassifier(n_estimators=150,
                                                       criterion='entropy',
                                                      max_depth=50,
                                                      max_features=20,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
r1.mean()

0.8122354497354497

### AdaBoost

In [42]:
from sklearn.ensemble import AdaBoostClassifier

In [102]:
AdaBoostClassifier?

In [101]:
ada = AdaBoostClassifier(n_estimators=10,learning_rate=1,random_state=100)

In [99]:
ada.fit(X_train_tf,Y_train)
Y_train_pred_ada = ada.predict(X_train_tf)
accuracy_score(Y_train,Y_train_pred_ada)

0.5089285714285714